<a href="https://colab.research.google.com/github/alexttrt/team5_mobilenetv3/blob/main/Team5_linebot_v3_ipnyb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install line-bot-sdk flask_ngrok
!pip install pyngrok
!ngrok authtoken 2SIRRD9b7E9IGNhqxvd4HNlyuP8_2wekze8q4VQJHXKKb2HbS
!gdown --id 16I0VF3o270kr6EbbOTHG0G62nFG281Go --output resnet50_model.pth
!pip install pymysql
!gdown --id 1jXWS2tPVgSdqFHaJmokyEh2Lc4vtggVL --output docker所需檔案.zip
!unzip -q docker所需檔案.zip
!pip install pygsheets

In [ ]:
from zipfile import ZipFile
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, request, abort, jsonify

from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models.events import (FollowEvent)
from linebot.models.template import(ButtonsTemplate)
from linebot.models import *

from torch import nn
from torchvision import models, transforms
from torchsummary import summary
from PIL import Image

import urllib.request, json, torch, io, torchvision, json, pymysql

from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
# 辨識圖片預處理
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # 調整圖片大小
    transforms.ToTensor(),  # 將圖片轉為 Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 正規化圖片
])

# Function模型辨識
def classify(image):
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)

    # 將圖片放入模型預測
    model = torch.load('/content/resnet50_model.pth', map_location=torch.device('cpu'))
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    input_batch = input_batch.to(device)
    with torch.no_grad():
      output = model(input_batch)
    percent = str(int(nn.Softmax(dim=1)(output).max().item()*100))+"%"
    predicted_idx = torch.max(output, 1)
    predicted_class = predicted_idx[1].item()
    predicted_label = labels[predicted_class]
    return predicted_label, percent

In [ ]:
def crawler():
  # df= pd.DataFrame()
  x = []
  url = 'https://wepet.tw/%E5%AF%B5%E7%89%A9%E9%A0%98%E9%A4%8A?page=1'
  headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
          ,"Referer":"https://wepet.tw/","Content-Type":"application/json; charset=UTF-8"}
  html=requests.get(url, headers=headers)
  response=BeautifulSoup(html.text)
  pet_datas= response.find_all("a",class_="text-white text-capitalize d-block text-truncate")
  for name_data in pet_datas:
    try:
      sub_url= name_data["href"]
      data=requests.get(sub_url, headers=headers)
      response2=BeautifulSoup(data.text)

      pet_detail=response2.find("div", class_="row font-size-15 mb-3")
      species= pet_detail.find_all("div", class_="col-12 mb-1")
      if "貓" in species[1].text:
        img=response2.find("img", class_="img-fluid")["src"]
        detail=response2.find("div", class_="col-12 mt-6 font-size-18").text
        detailx=[]
        addition_data=[["名稱",name_data.text],["圖片",img],["資訊網址",sub_url],["詳述",detail]]
        detailx.extend(addition_data)
        for i in species:
          detailx.append(i.text.split("："))
        x.append(detailx[0:13])
        # df= df.append([x], ignore_index=True)
      else:
        pass
    except:
        print("處理錯誤")
  return x

In [ ]:
x = crawler()
x

[[['名稱', '珍珠奶茶與小酒窩'],
  ['圖片',
   'https://wepet.tw/storage/adopt/adopt_230815201551952113819_4093201751.jpg'],
  ['資訊網址', 'https://wepet.tw/領養資訊-230815201551952113819'],
  ['詳述',
   '🐯待領養專區🐯\n\r\n端午寶寶\r\n一起送養，不拆送（都是公貓）\r\n可親送，需接受家訪，台南地區佳\r\n目前貓在台南安平區'],
  ['品種', '虎斑/白底虎斑'],
  ['種類', '貓'],
  ['性別', '公'],
  ['體型', '小型'],
  ['毛色', '虎斑、白底虎斑'],
  ['年紀', '幼年'],
  ['結紮', '未結紮'],
  ['所在地', '台南市 安平區'],
  ['刊登日期', '2023-08-15']],
 [['名稱', '憨吉'],
  ['圖片',
   'https://wepet.tw/storage/adopt/adopt_230814030038415713812_8552853238.jpg'],
  ['資訊網址', 'https://wepet.tw/領養資訊-230814030038415713812'],
  ['詳述', '原飼主長期在外工作無法給予好的陪伴與照顧'],
  ['品種', '米克斯'],
  ['種類', '貓'],
  ['性別', '母'],
  ['體型', '小型'],
  ['毛色', '白底虎斑'],
  ['年紀', '幼年'],
  ['結紮', '未結紮'],
  ['所在地', '高雄市 左營區'],
  ['刊登日期', '2023-08-14']],
 [['名稱', '麻糬'],
  ['圖片',
   'https://wepet.tw/storage/adopt/adopt_230813111550822513811_5598196550.jpg'],
  ['資訊網址', 'https://wepet.tw/領養資訊-230813111550822513811'],
  ['詳述',
   '*健康狀態：良好，食慾正常愛喝水\r\n*個性：親人愛撒嬌，不挑食會呼

In [ ]:
# Function連接資料庫
import numpy as np

def cat_mysql(cat):
  c1 = pymysql.connect(
    host='35.229.247.93',
    user='root',
    passwd='mysqlpassword',
    database='my_database',
    charset='utf8',
    port=3306,
  )
  p1 = c1.cursor()
  p1.execute("SELECT * FROM `catdata02` WHERE `species`='{}'".format(cat))
  x = p1.fetchall()

  data = []
  for i in range(3):
    idx = np.random.randint(len(x))
    data.append(x[idx])
  return data

In [ ]:
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

with open("env.json") as f :
  env = json.load(f)

with open("petset_labels.txt", "r") as f:
  labels = (f.read().split())

line_bot_api = LineBotApi(env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
label = env['YOUR_LABELS']

<ipython-input-33-ba06e731ba5e>:10: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(env['YOUR_CHANNEL_ACCESS_TOKEN'])
<ipython-input-33-ba06e731ba5e>:11: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])


In [ ]:
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    print(body)
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

In [ ]:
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    with open("/content/drive/MyDrive/users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')
    line_bot_api.reply_message(
    event.reply_token,
    [TextSendMessage("歡迎加入Meowow AI!\n這裡提供各式便捷的服務\n快點來體驗看看吧\n請輸入@打開主選單\n或直接使用下方快捷鍵!"),
     StickerSendMessage(package_id="1070", sticker_id="17839"),
     quick_reply]
    )


In [ ]:
# 收到圖片時 訊息處理
@handler.add(MessageEvent, message=ImageMessage)
def handle_img_message(event):
  message_id = event.message.id
  message_content = line_bot_api.get_message_content(message_id)

  b = b''  # 轉成byte資料形式
  for chunk in message_content.iter_content():
    b += chunk
  img = Image.open(io.BytesIO(b))
  result, percent= classify(img)

  ans = open("/content/ttt.txt", "w")
  ans.write(result)
  ans.close()

  correct = open("/content/kkk.txt", "w")
  correct.write(percent)
  correct.close()
  # 回應訊息
  line_bot_api.reply_message(event.reply_token,[TextSendMessage(text="圖片已上傳囉!"),confirm4])


In [ ]:
#處理模板按鈕的訊息
@handler.add(PostbackEvent)
def handle_post_message(event):
  user_profile = line_bot_api.get_profile(event.source.user_id)

  if (event.postback.data.find('$') != -1):
    line_bot_api.reply_message(event.reply_token,
                               main_template_dict.get(event.postback.data))

  elif (event.postback.data.find('!') != -1):
    line_bot_api.reply_message(event.reply_token,
                               breakingnews_dict.get(event.postback.data))

  elif (event.postback.data.find('@') != -1):
    line_bot_api.reply_message(event.reply_token,
                               adoptprocess_dict.get(event.postback.data))

  elif (event.postback.data.find('#') != -1):
    line_bot_api.reply_message(event.reply_token,
                               img_choice_dict.get(event.postback.data))

  elif (event.postback.data.find('*貓貓辨識') != -1):
    with open("ttt.txt", 'r', encoding="utf-8") as x:
      kk = x.read()
    with open("kkk.txt", 'r', encoding="utf-8") as y:
      rr = y.read()

    data = cat_mysql(kk)

    io = []
    for i in range(3):
      if len(data[i][13])>20 :
        x = data[i][13][0:21] + "...更多"
        io.append(x)
      else:
        x = data[i][13] + "...更多"
        io.append(x)

    similar_cat = TemplateSendMessage(
      alt_text='CarouselTemplate',
      template=CarouselTemplate(columns=[
        CarouselColumn(thumbnail_image_url=data[0][11],
                       title=data[0][1] + data[0][2],
                       text=io[0],
                       actions=[MessageAction(label='更多資訊', text='更多資訊網址:'+data[0][10])]),
        CarouselColumn(thumbnail_image_url=data[1][11],
                       title=data[1][1] + data[1][2],
                       text=io[1],
                       actions=[MessageAction(label='更多資訊', text='更多資訊網址:'+data[1][10])]),
        CarouselColumn(thumbnail_image_url=data[2][11],
                       title=data[2][1] + data[2][2],
                       text=io[2],
                       actions=[MessageAction(label='更多資訊', text='更多資訊網址:'+data[2][10])])
      ]))
    line_bot_api.reply_message(event.reply_token, [
      TextSendMessage(text=f"辨識結果為:{kk}\n準確率:{rr}"),
      TextSendMessage(text="稍後為您列出相似的待收養浪貓"), similar_cat, quick_reply2
    ])

  elif (event.postback.data.find('postback1') == 0):
    line_bot_api.reply_message(event.reply_token, main_temp)

  elif (event.postback.data.find('action1') == 0):

    x = crawler()
    unadopted_cat = TemplateSendMessage(
      alt_text='ImageCarouselTemplate',
      template=ImageCarouselTemplate(columns=[
        ImageCarouselColumn(
          image_url=
          x[0][1][1],
          action=MessageAction(label=x[0][0][1], text="資訊網址:"+x[0][2][1]+"\n刊登時間:"+x[0][12][1])),
        ImageCarouselColumn(
          image_url=
          x[1][1][1],
          action=MessageAction(label=x[1][0][1], text="資訊網址:"+x[1][2][1]+"\n刊登時間:"+x[1][12][1])),
        ImageCarouselColumn(
          image_url=
          x[2][1][1],
          action=MessageAction(label=x[2][0][1], text="資訊網址:"+x[2][2][1]+"\n刊登時間:"+x[2][12][1])),
        ImageCarouselColumn(
          image_url=
          x[3][1][1],
          action=MessageAction(label=x[3][0][1], text="資訊網址:"+x[3][2][1]+"\n刊登時間:"+x[3][12][1]))
      ]))

    line_bot_api.reply_message(event.reply_token, [TextSendMessage(text="資料連結完成"),unadopted_cat,quick_reply2])
  else:
    pass


In [ ]:
quick_reply = TextSendMessage(
  text='快速選單',
  quick_reply=QuickReply(items=[
    QuickReplyButton(action=PostbackAction(label="主選單", data="postback1")),
    QuickReplyButton(action=MessageAction(label="最新消息", text="更多流程")),
    QuickReplyButton(action=CameraRollAction(label="圖片辨識")),
    QuickReplyButton(action=CameraAction(label="打開相機")),
  ]))
quick_reply2 = TextSendMessage(
  text='繼續其它服務',
  quick_reply=QuickReply(items=[
    QuickReplyButton(action=PostbackAction(label="主選單", data="postback1")),
    QuickReplyButton(action=PostbackAction(label="訊息記錄", data="$action3")),
    QuickReplyButton(action=MessageAction(label="最新消息", text="更多流程")),
  ]))

In [ ]:
north=["台北市","新北市","基隆市","桃園市","新竹市","新竹縣","宜蘭縣"]
mid=["苗栗縣","台中市","彰化縣","南投縣","雲林縣"]
south=["嘉義市","嘉義縣","台南市","高雄市","屏東縣","澎湖縣"]

northdistrict = TemplateSendMessage(
  alt_text='ImageCarouselTemplate',
  template=ImageCarouselTemplate(columns=[
    ImageCarouselColumn(
      image_url=
      'https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/326650117_863447528222413_3496208209142595679_n.png?_nc_cat=102&ccb=1-7&_nc_sid=09cbfe&_nc_ohc=El5Mz-zggQoAX_ry1gS&_nc_ht=scontent-tpe1-1.xx&oh=00_AfDtHpmSq38bAUjI4hIWc0SVGYXsH9nZByoMGWyVcqHR4Q&oe=64AEDB08',
      action=URIAction(label="臺北市動物之家",
                       uri='https://www.facebook.com/tcapo.tas/')),
    ImageCarouselColumn(
      image_url=
      'https://www.ahiqo.ntpc.gov.tw/upload/cht/article/72025406353f19414d1e46361db0417b.jpg',
      action=URIAction(label='新北市政府動物保護防疫處',
                       uri='https://www.ahiqo.ntpc.gov.tw/cht/index.php?')),
    ImageCarouselColumn(
      image_url='https://taw.tycg.gov.tw/images/TOP-1170.jpg',
      action=URIAction(label='桃園市動物保護教育園區',
                       uri="https://taw.tycg.gov.tw/X00_index.aspx")),
    ImageCarouselColumn(
      image_url=
      'https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/357736134_6619687564719543_2303349824690039417_n.jpg?_nc_cat=110&ccb=1-7&_nc_sid=730e14&_nc_ohc=bA5xejT3dPYAX-1NyA8&_nc_ht=scontent-tpe1-1.xx&oh=00_AfAZPihpN06QDvoqtBtXmHSUyZdHpG1kpbYy9sjX_I6g7Q&oe=64AE35CB',
      action=URIAction(label='宜蘭縣流浪動物中途之家',
                       uri="https://animal.e-land.gov.tw/Default.aspx")),
    ImageCarouselColumn(
      image_url=
      'https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/357732749_6619692518052381_422083509601730492_n.jpg?_nc_cat=100&ccb=1-7&_nc_sid=730e14&_nc_ohc=f5SbWUZIWxUAX8f0MW8&_nc_ht=scontent-tpe1-1.xx&oh=00_AfBehFRYZVmFjQWRcN0hSH168fZzgA9xOcp5rpxtycO7dw&oe=64AE1B43',
      action=URIAction(
        label='新竹市動物保護教育園區',
        uri="https://puppy.hccg.gov.tw/ch/home.jsp?id=6&parentpath=0")),
    ImageCarouselColumn(
      image_url=
      'https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/358966186_6619692448052388_8845866107030886513_n.jpg?_nc_cat=103&ccb=1-7&_nc_sid=730e14&_nc_ohc=wrUHEPo5QhwAX-sYFoR&_nc_oc=AQkrbzcDYIZCG_3CbpZIUwlZYtAgIfAVM0SVnvktWiXGsErWU4fEv9WPVGsC3qtvpkM&_nc_ht=scontent-tpe1-1.xx&oh=00_AfBIlQ_HJ1MimPmjzA76so3dGkw_OgcfMyZO3LSS1ZQGFQ&oe=64AE29FC',
      action=URIAction(
        label='基隆市寵物銀行',
        uri="https://pet234.com/shop-a156f2e9-06db-4310-9b13-d1b4a2f42518.html"
      ))
  ]))

middistrict = TemplateSendMessage(
  alt_text='ImageCarouselTemplate',
  template=ImageCarouselTemplate(columns=[
    ImageCarouselColumn(
      image_url='https://img.onl/KegNsz',
      action=URIAction(
        label='臺中市動物之家南屯園區',
        uri="https://www.animal.taichung.gov.tw/1521490/Lpsimplelist")),
    ImageCarouselColumn(
      image_url='https://img.onl/uafqDU',
      action=URIAction(
        label='苗栗縣生態保育教育中心',
        uri="https://www.ahiqo.ntpc.gov.tw/cht/index.php?")),
    ImageCarouselColumn(
      image_url='https://img.onl/EM6Z3U',
      action=URIAction(
        label='彰化縣流浪狗中途之家',
        uri="https://www.chcgadcc.gov.tw/")),
    ImageCarouselColumn(
      image_url=
      'https://img.onl/E5pN0D',
      action=URIAction(
        label='南投縣公立動物收容所',
        uri=
        "http://adcc.nantou.gov.tw/introduce/leading.asp?id=%7BAAF1647F-051F-47C0-BCFC-03E9C497E4D2%7D"
      ))
  ]))

southdistrict = TemplateSendMessage(
  alt_text='ImageCarouselTemplate',
  template=ImageCarouselTemplate(columns=[
    ImageCarouselColumn(
      image_url=
      'https://img.onl/aQRsEU',
      action=URIAction(
          label='臺南市動物之家灣裡站',
          uri="https://ahipo.tainan.gov.tw/Default.aspx")),
    ImageCarouselColumn(
      image_url=
      'https://img.onl/mtLpOG',
      action=URIAction(
          label='雲林縣流浪動物收容所',
          uri="https://livestock.yunlin.gov.tw/News.aspx?n=910&sms=9439")),
    ImageCarouselColumn(
      image_url=
      'https://img.onl/Tfhqdb',
      action=URIAction(
          label='嘉義縣流浪犬中途之家',
          uri=
           "https://www.facebook.com/pages/category/Interest/%E9%9D%9E%E5%AE%98%E6%96%B9%E5%98%89%E7%BE%A9%E7%B8%A3%E6%B0%91%E9%9B%84%E6%94%B6%E5%AE%B9%E6%89%80%E6%B5%81%E6%B5%AA%E7%8A%AC%E4%B8%AD%E9%80%94%E4%B9%8B%E5%AE%B6-1069116556450723/"
      )),
    ImageCarouselColumn(
      image_url=
      'https://img.onl/K3XaWs',
      action=URIAction(
          label='屏東縣公立犬貓中途之家',
          uri="https://www.facebook.com/adoptmeloveme/"))
  ]))


In [ ]:
# 主選單建立
main_temp = TemplateSendMessage(
  alt_text='ButtonsTemplate',
  template=ButtonsTemplate(
    thumbnail_image_url=
    'https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/357743504_6618450718176561_8982136937443641572_n.jpg?_nc_cat=109&ccb=1-7&_nc_sid=730e14&_nc_ohc=nGvVf9HBygEAX_-V2Ln&_nc_ht=scontent-tpe1-1.xx&oh=00_AfCBiQkBPIuhz3xeJM-bQ-XrQUO4B4iEifV1nIa7iZbAiw&oe=64D5820F',
    title='MeowowAI 主選單',
    text='快來體驗便捷的領養功能吧!    以下為四大主要功能',
    actions=[
      PostbackAction(label='最新消息', text="新鮮資訊不漏接!", data='$action1'),
      PostbackAction(label='我要認養', text="我想了解領養流程", data='$action2'),
      PostbackAction(label='寵物日常', text="記錄愛貓的生活", data='$action3'),
      PostbackAction(label='出去遛遛', text="愛貓外出不設限~", data='$action4')
    ]))

# 副選單模板(1)
news_temp = TemplateSendMessage(
  alt_text='ButtonsTemplate',
  template=ButtonsTemplate(
    thumbnail_image_url=
    "https://www.supercoloring.com/sites/default/files/styles/coloring_full/public/cif/2019/11/cat-broadcast-announcer-coloring-page.png",
    title='寵物界各種大小事',
    text='最新資訊都在這裡',
    actions=[
      PostbackAction(label="走失協尋", text="我要通報!", data='!action2'),
      PostbackAction(label="新增收容", text="可領養的喵喵", data='action1'),
      URIAction(label="熱賣商品", uri="https://petpetgo.com/")
    ]))

# 副選單模板(2)
adoptprocess_template = TemplateSendMessage(
  alt_text='ButtonsTemplate',
  template=ButtonsTemplate(
    thumbnail_image_url=
    "https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/358391254_6618537664834533_1338829207264939757_n.jpg?_nc_cat=108&ccb=1-7&_nc_sid=730e14&_nc_ohc=-CpG6AtT2PUAX9PMsE-&_nc_ht=scontent-tpe1-1.xx&oh=00_AfB35RlZ0BQdeqtFk6eFKpwXPytjUhf-3jB1-i-Ui_M_nQ&oe=64D5D74C",
    title="呼籲以領養代替購買喔!",
    text="基本領養需知:提供領養所需資訊\n進入領養程序:透過貓咪圖片辨識,快速配對\n領養問卷調查:了解用戶領養意願",
    actions=[
      PostbackAction(label="基本領養需知", text="我想了解領養前要有甚麼準備?", data='@action1'),
      PostbackAction(label="進入領養程序", text="我已了解領養流程 開始領養程序", data='@action2'),
      URIAction(label="領養問卷調查", uri="https://forms.gle/aXWCM6kbH9TS1u3n7")
    ]))

# 副選單模板(3)
petsdaily_template = TemplateSendMessage(
  alt_text='ButtonsTemplate',
  template=ButtonsTemplate(
    thumbnail_image_url=
    "https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/359324996_6619608981394068_5502915817996258193_n.jpg?_nc_cat=103&ccb=1-7&_nc_sid=730e14&_nc_ohc=OOq56DketeAAX9SbxB_&_nc_ht=scontent-tpe1-1.xx&oh=00_AfA-CXbrzkJx49WG9BOcA69pdmGY-yFuIPe6BCQx-swCVw&oe=64AE9E07",
    title='毛小孩的生活管家',
    text='毛小孩不會講話...記得多關心她快速記錄愛貓基本資訊!',
    actions=[
      MessageAction(
        label="活動紀錄",
        text="這裡的紀錄會記在google sheet裡",
      ),
      MessageAction(
        label='飲食記錄',
        text="這裡的紀錄會記在google sheet裡",
      ),
      MessageAction(
        label="病歷紀錄",
        text="這裡的紀錄會記在google sheet裡",
      ),
      MessageAction(
        label="獸醫查詢",
        text="https://goo.gl/maps/AoD7LWi4XYvhqewD8",
      )
    ]))

# 副選單模板(4)
outdoor_template = TemplateSendMessage(
  alt_text='ButtonsTemplate',
  template=ButtonsTemplate(
    thumbnail_image_url=
    "https://scontent-tpe1-1.xx.fbcdn.net/v/t39.30808-6/359351398_6618907874797512_6932741876875671605_n.jpg?_nc_cat=110&ccb=1-7&_nc_sid=730e14&_nc_ohc=kHZxjf621fkAX9oQ9gA&_nc_ht=scontent-tpe1-1.xx&oh=00_AfB3FY2Q62ITjCu9sVuOr0T9oN_NYydGjOUUaLPVhoBWsQ&oe=64AF3C9F",
    title='帶毛小孩出門曬太陽',
    text='不知道哪裡有寵物友善空間嗎?趕緊來查詢!點擊下列按鈕即可以當前位置搜尋鄰近的地圖資訊!',
    actions=[
      URIAction(label="寵物友善餐廳",
                uri="https://goo.gl/maps/X5E4rmk3PtiRyHTv8"),
      URIAction(label="寵物旅館",
                uri="https://goo.gl/maps/YoLaPmaWEpcjqyeRA"),
      URIAction(label="寵物公園",
                uri="https://goo.gl/maps/eBF3LDrhD8jnP1sb6")
    ]))

In [ ]:
#確認選單
confirm = TemplateSendMessage(
  alt_text='ConfirmTemplate',
  template=ConfirmTemplate(text='以上為領養前準備',
                           actions=[
                             MessageAction(label='已了解', text="我已經完全了解!"),

                           ]))

confirm2 = TemplateSendMessage(
  alt_text='ConfirmTemplate',
  template=ConfirmTemplate(text='請問理想的貓咪類型?',
                           actions=[
                             URIAction(
                               label='上傳圖片辨識',
                               uri='https://line.me/R/nv/cameraRoll/single'),
                             MessageAction(
                               label='目前收容清單',
                               text="我想先看看待收容的貓咪",
                             ),
                           ]))
confirm3 = TemplateSendMessage(
  alt_text='ConfirmTemplate',
  template=ConfirmTemplate(text='以上是見到本喵時要確認的事情',
                                 actions=[
                                   PostbackAction(label="已了解",
                                                  text="我準備好了!帶我開始領養程序吧",
                                                  data='@action2'),


                                 ]))
confirm4 = TemplateSendMessage(
  alt_text='ConfirmTemplate',
  template=ConfirmTemplate(text='請選擇上傳圖片用途',
                           actions=[
                             PostbackAction(label="走失通報",
                                            text="幫我上傳到走失通報紀錄",
                                            data='#走失通報'),
                             PostbackAction(label='貓咪辨識',
                                            text="與我有緣的貓星人快來吧!",
                                            data='*貓貓辨識'),
                           ]))

In [ ]:
missing_process = TextSendMessage(text="請上傳照片及回傳寵物特徵描述 (訊息前面加!協尋)")
finding_process = TextSendMessage(text="請上傳照片及回傳尋獲地址或您的聯絡方式 (訊息前面加!尋獲)")

# 走失通報選單
missing_confirm = TemplateSendMessage(alt_text='ConfirmTemplate',
                                      template=ConfirmTemplate(
                                        text='請選擇通報種類',
                                        actions=[
                                          MessageAction(
                                            label='寵物走失',
                                            text="我家毛小孩走失了 麻煩大家幫我找找!",
                                          ),
                                          MessageAction(
                                            label='路上尋獲',
                                            text="我好像看到走失的貓貓",
                                          ),
                                        ]))

In [ ]:
# 收到用戶發出文字消息時，按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):

    if (event.message.text in north):
        line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage(text="下列是靠近您居住地的收容所"),
        northdistrict,
         quick_reply])

    elif(event.message.text in mid):
        line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage(text="下列是靠近您居住地的收容所"),
        middistrict,
         quick_reply])

    elif(event.message.text in south):
        line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage(text="下列是靠近您居住地的收容所"),
        southdistrict,
         quick_reply])

    elif(event.message.text == "@"):
        line_bot_api.reply_message(
        event.reply_token,main_temp)
    elif(event.message.text == "照片辨識"):
        line_bot_api.reply_message(
        event.reply_token,TextSendMessage(text="請選擇一張照片上傳"))

    elif(event.message.text == "我已經完全了解!"):
        line_bot_api.reply_message(
        event.reply_token,
        [notice1,notice2,confirm3])

    elif(event.message.text == "更多流程"):
        line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage(text="讓我幫您查詢附近的收容所"),
         TextSendMessage(text="請輸入您的居住縣市")])

    elif(event.message.text == "我想先看看待收容的貓咪"):
        line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage(text="讓我幫您查詢附近的收容所"),
         TextSendMessage(text="請輸入居住縣市")])

    elif(event.message.text == "重新上傳"):
        line_bot_api.reply_message(
        event.reply_token,
        confirm2)
    elif(event.message.text == "主選單"):
        line_bot_api.reply_message(
        event.reply_token,
        main_temp)
    elif(event.message.text == "我家毛小孩走失了 麻煩大家幫我找找!"):
        line_bot_api.reply_message(
        event.reply_token,
        missing_process)
    elif(event.message.text == "我好像看到走失的貓貓"):
        line_bot_api.reply_message(
        event.reply_token,
        finding_process)

    elif(event.message.text.find('！協尋')!= -1):

      line_bot_api.reply_message(
            event.reply_token,
                [TextSendMessage(text="已將內容記錄到筆記本 若要查看可使用!查看協尋紀錄"),quick_reply2]
            )
      with open("協尋紀錄.txt","a")as file:
            file.write(event.message.text+"\n")
            file.write("======"*3+"\n")

    elif(event.message.text.find('#查看協尋紀錄')!= -1):
      with open("協尋紀錄.txt","r",encoding="utf-8") as file:
         text1=file.read()
         line_bot_api.reply_message(
            event.reply_token,
                [TextSendMessage(text="以下是協尋紀錄內容"),TextSendMessage(text=text1)]
            )
    elif(event.message.text.find('！尋獲')!= -1):
      line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(text="已記錄您的回報內容 會盡快通知飼主 感謝您的熱心")
            )
      with open("回報記錄.txt","a")as file:
            file.write(event.message.text+"\n")
            file.write("======"*3+"\n")
    else:
      pass

In [ ]:
#領養貓咪前的準備
section1=TextSendMessage("決定要領養幼貓還是成貓\n"+"-"*22+"\n"+'''認養貓咪前最重要的一個決定就是要選擇成貓還是幼貓了！
雖然大多數人一開始可能會因為小貓可愛的模樣而想認養幼貓。
但其實認養幼貓不僅需要在食物、保暖、醫療上花更多的心力和時間照顧，
更因為小貓個性還不穩定容易有咬人、抓人、破壞家具等行為而必須額外訓練！
因此若是沒有經驗、時間比較少又沒有特別想要小貓的話
建議還是領養成貓比較適合！
''')

section2=TextSendMessage("確定自己有時間、地點和金錢照顧貓咪\n"+"-"*22+"\n"+'''貓咪的壽命可以長達 10 – 14 歲，
因此在領養前要先確定自己可以在那麼長的時間內
都可以負擔起照顧他們的責任。現在的工作有時間陪伴他們嗎？
之後有可能換成比較忙的工作，甚至出國嗎？租屋的話可否養寵物？
之後搬家的話呢？另外貓咪的飼料、貓砂、醫療等一個月需要花上 3000 – 5000 塊，
現在的經濟情況是否允許？家人是否同意等等都是需要仔細思考的問題！
''')

section3=TextSendMessage("多了解關於照顧貓咪文章和內容\n"+"-"*22+"\n"+'''在領養貓咪前，建議最好先了解一些例如：
貓咪該吃什麼？什麼不能吃？貓砂怎麼換？貓咪需要洗澡嗎？
等等照顧貓咪的基本知識。不僅可以讓你做好心理準備，
帶回貓咪後也比較不會手忙腳亂。
''')

#貓咪認養時的注意事項
notice1=TextSendMessage("詢問貓咪的身體狀況\n"+"-"*22+"\n"+'''認養貓咪前一定要先確認貓咪的健康狀況！
詢問送養者貓咪的年齡、性別、是否結紮、有無先天或後天疾病、疫苗接種狀況等等，
一般情形下送養者都會確實告知。建議沒有經驗的話盡量不要養本身就有疾病的貓，
會需要非常多額外的心力。若真的要養病貓的話也建議領養前多學習相關的照護知識！
''')

notice2=TextSendMessage("事前熟悉貓咪的個性\n"+"-"*22+"\n"+'''跟人一樣，每隻貓咪也有自己的個性。有些就是比較親人愛撒嬌，
有些則是看到你就一溜煙跑的不見貓影。貓咪的個性是不太會改變的，
所以如果想要跟自己的貓咪有點親密時光，不想養得很ㄘㄟˋ心的話，
建議可以的話在領養前實際跟貓咪玩玩，相處一段時間 ，確定貓咪的親人程度再進行領養！
''')

In [ ]:
main_template_dict = {
  "$action1": news_temp,
  "$action2": adoptprocess_template,
  "$action3": petsdaily_template,
  "$action4": [outdoor_template, quick_reply2]
}
breakingnews_dict = {"!action2": missing_confirm}
adoptprocess_dict = {
  "@action1": [section1, section2, section3, confirm],
  "@action2": [TextSendMessage(text="感謝您的愛心 讓我帶領你走一趟流程吧!"), confirm2]
}
img_choice_dict = {"#走失通報": [TextSendMessage(text="圖片已上傳"), quick_reply2]}


In [ ]:
# 主程序運行
port = 5000
# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))
app.run()